# M/G/1 model

- simulator taken from https://github.com/mackelab/SNL_py3port, which contains the original https://github.com/gpapamak/snl after 2to3 conversion with minimal edits (deactivating generator-internal summary stats normalization).


In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import timeit

import snl
import snl.simulators.mg1 as sim

import delfi.generator as dg
import delfi.distribution as dd
from delfi.utils.viz import plot_pdf
import delfi.inference as infer

seed = 42

# SNPE parameters

# training schedule
n_train=1000
n_rounds=1

# fitting setup
minibatch=100
epochs=300

# network setup
n_hiddens=[50,50]
reg_lambda=0.01

# convenience
pilot_samples=1000
svi=False
verbose=True
prior_norm=False

# SNPE-C parameters
n_null = 1

# MAF parameters
mode='random' # ordering of variables for MADEs
n_mades = 5 # number of MADES
act_fun = 'tanh'
batch_norm = False # batch-normalization currently not supported
train_on_all = True # now supported feature

# MDN parameters
n_components = 1

In [ ]:
from delfi.simulator import BaseSimulator


def init_g(seed):
    # prior 

    #prior = sim.Prior() # proposal will happily sample from outside the prior unless it's dd.Uniform !
    prior = dd.Uniform(lower=[ 0, 0,  0  ], 
                       upper=[10,20,1./3.])

    # model 
    model_snl = sim.Model()
    class MG1(BaseSimulator):
        """M/G/1 simulator

        Parameters
        ----------
        dim : int
            Number of dimensions of parameters
        seed : int or None
            If set, randomness is seeded
        """        

        def gen_single(self, params):
            """ params = (lower bound of server processing time,
                          upper bound of server processing time,
                          rate customer arrivals )

            """
            return model_snl.sim(params)

    model = MG1(dim_param=3)

    # summary statistics
    summary = sim.Stats()

    # generator
    g = dg.Default(prior=prior, model=model, summary=summary, seed=seed+41)
    return g

In [ ]:
g = init_g(seed=seed)

pars_true = np.array([1, 5, 0.2])  # taken from SNL paper
obs = g.model.gen_single(pars_true)  # should also recover
obs_stats = g.summary.calc([obs])    # xo from SNL paper !

In [ ]:

if train_on_all:
    epochs = [epochs//(r+1) for r in range(n_rounds)]

# control MAF seed
rng = np.random
rng.seed(seed)

# generator
g = init_g(seed=seed)

# inference object
res_C = infer.SNPEC(g,
                 obs=obs_stats,
                 n_hiddens=n_hiddens,
                 seed=seed,
                 reg_lambda=reg_lambda,
                 pilot_samples=pilot_samples,
                 svi=svi,
                 n_mades=n_mades, # providing this argument triggers usage of MAFs (vs. MDNs)
                 act_fun=act_fun,
                 mode=mode,
                 rng=rng,
                 batch_norm=batch_norm,
                 verbose=verbose,
                 prior_norm=prior_norm)

# train
t = timeit.time.time()

logs_C, tds_C, posteriors_C = res_C.run(
                    n_train=n_train,
                    proposal='discrete',
                    moo='resample',
                    n_null = n_null,
                    n_rounds=n_rounds,
                    train_on_all=train_on_all,
                    minibatch=minibatch,
                    epochs=epochs)

print(timeit.time.time() - t)


In [ ]:
for r in range(n_rounds):
    plt.plot(logs_C[r]['loss'])
    plt.show()

In [ ]:
for r in range(len(logs_C)):
    
    posterior_C = posteriors_C[r]
    #posterior_C.ndim = posterior_A.ndim
    
    fig,_=plot_pdf(dd.Gaussian(m=np.zeros(pars_true.size), S=1e-15*np.eye(pars_true.size)), 
                   samples=posterior_C.gen(1000).T,
                   gt=pars_true, 
                   #lims=[[0,10],[0,20],[0., 1./3.]],
                   #lims=[0,10],
                   resolution=100,
                   ticks=True,
                   figsize=(16,16));
    
    fig.suptitle('SNPE-C posterior estimates, round r = '+str(r+1), fontsize=14)
    print('negative log-probability of ground-truth pars \n', -posterior_C.eval(pars_true, log=True))

## non-accumulative training

In [ ]:
for r in range(len(logs_C)):
    
    posterior_C = posteriors_C[r]
    #posterior_C.ndim = posterior_A.ndim
    
    fig,_=plot_pdf(dd.Gaussian(m=np.zeros(pars_true.size), S=1e-15*np.eye(pars_true.size)), 
                   samples=posterior_C.gen(1000).T,
                   gt=pars_true, 
                   #lims=[[0,10],[0,20],[0., 1./3.]],
                   #lims=[0,10],
                   resolution=100,
                   ticks=True,
                   figsize=(16,16));
    
    fig.suptitle('SNPE-C posterior estimates, round r = '+str(r+1), fontsize=14)
    print('negative log-probability of ground-truth pars \n', -posterior_C.eval(pars_true, log=True))